### Melon 100 Chart 스크래핑
* 100곡의 노래의 제목과 Song ID 파싱
* Song ID로 상세페이지로 100번 요청을 해서 상세정보를 파싱
* 100곡의 상세정보를 json file에 저장
* json file을 표데이터를 만들기
* 표데이터를 DB에 저장

In [17]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
req_header = {
    #browser 정보
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
}
res = requests.get(url, headers=req_header)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select("div#tb_list"))) #1개
    print(len(soup.select("div#tb_list tr"))) #101개
    print(len(soup.select("div#tb_list tr div.wrap_song_info"))) #200개
    print(len(soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']"))) #100개
    
    atag_list = soup.select("div#tb_list tr div.wrap_song_info a[href*='playSong']")
    for idx,atag in enumerate(atag_list,1):
        title = atag.text
        #print(f'{idx}위, {title}')
        print(atag['href'])

200
1
101
200
100
javascript:melon.play.playSong('1000002721',34041584);
javascript:melon.play.playSong('1000002721',33658563);
javascript:melon.play.playSong('1000002721',33480898);
javascript:melon.play.playSong('1000002721',33998510);
javascript:melon.play.playSong('1000002721',33487342);
javascript:melon.play.playSong('1000002721',33725775);
javascript:melon.play.playSong('1000002721',32698101);
javascript:melon.play.playSong('1000002721',33666269);
javascript:melon.play.playSong('1000002721',33625988);
javascript:melon.play.playSong('1000002721',33507137);
javascript:melon.play.playSong('1000002721',33655994);
javascript:melon.play.playSong('1000002721',33496587);
javascript:melon.play.playSong('1000002721',33623210);
javascript:melon.play.playSong('1000002721',33359317);
javascript:melon.play.playSong('1000002721',32872978);
javascript:melon.play.playSong('1000002721',33691794);
javascript:melon.play.playSong('1000002721',33503722);
javascript:melon.play.playSong('1000002721',338